In [1]:
import pandas as pd

df = pd.read_csv('/Users/jaesolshin/Documents/GitHub/youtube_dashboard/petition.csv')

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Okt
okt = Okt()

In [ ]:
def tokenizer(doc):
    return [word for word in okt.nouns(doc) if len(doc) > 1]

vec = CountVectorizer(tokenizer=tokenizer,
                      max_df = 0.5, min_df= 5,
                      max_features= 1000)

pet_cv = vec.fit_transform(df.content)
print(pet_cv)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_component = 15,
                                n_jobs=-1,
                                random_state=42)

%time pet_topics = lda.fit_transform(pet_cv)

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d: " %topic_idx, end='')
        print(", ".join([feature_names[i]
                         for i in topic.argsort()[:-n_top_waords - 1: -1]]))
        
print_top_words(lda, vec.get_feature_names_out(), 15)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Okt
trend_data = pd.DataFrame(pet_topics, columns=['Topic'+str(i) for i in range(1,16)])
trend_data = pd.concat([trend_data, df.start_map(lambda x: x[:7], axis=1)])
trend_data.iloc[:5, -5:]

In [ ]:
trend = trend_data.groupby(['start']).mean()
trend.iloc[:5,-5:]

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

fix, axes = plt.subplots(5, 3, sharex='col', figsize=(12,16))
for col, ax in zip(trend.columns.tolist(), axes.ravel()):
    ax.set_title(col)
    ax.axes.xaxis.set_visible(False)
    ax.plot(trend[col])
plt.show()